# Repeated Bertrand, test notebook

In [1]:
from scipy.optimize import minimize, fsolve

In [2]:
from game_tournament.game import RepeatedBertrandGame, Tournament
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 

In [3]:
tournament_list = []

## Pure logit demand
$$ D_i(p_i, p_j) = \frac{\exp(1-p_i)}{1 + \exp(1-p_i) + \exp(1 - p_j)} $$

In [4]:
def demand_function(p1, p2, i): 
    u1 = np.exp(1.-1.0*p1)
    u2 = np.exp(1.-1.0*p2)
    u0 = 1.0 
    
    denom = u0 + u1 + u2
    
    if i == 0: 
        return u1/denom 
    elif i == 1: 
        return u2/denom
    else: 
        raise Exception(f'Cannot return demand for player i={i}: must be 0 or 1.')

Action space and cost function. 

In [5]:
c = 0.5
pmin = c
pmax = 10.

# Run a tournament 
This will 

1. read all *.py files in `player_file_path` and load a `player` class from each 
2. set up an all_play_all tournament between said players 
3. provide rankings (win = 2, draw = 1, loss = 0 points)

In [6]:
players_file_path = './players/'
tournament = Tournament(players_filepath=players_file_path, game=RepeatedBertrandGame, tournament_name='PriceComp')
print(tournament)

Tournament ready with 5 players


In [7]:
np.random.seed(1337)
beta = 1.0
tournament.start_tournament(T=100, beta=beta, demand_function=demand_function, marginal_cost=1.0, price_range=(pmin, pmax), discount_factor=beta)

100%|██████████| 10/10 [00:00<00:00, 160.67it/s]


Top placements are:
                   Name  Points
3  Undercutting Bastard     8.0
1               Randawg     6.0
4        Tit-for-tatter     3.0
2                  PMAX     3.0
0                  pmin     0.0


Show the total rankings of the game

In [8]:
tournament.tournament_rank

,Name,Points
3,Undercutting Bastard,8.0
1,Randawg,6.0
4,Tit-for-tatter,3.0
2,PMAX,3.0
0,pmin,0.0


# Undercutting motive

$$
u_{ij} = \begin{cases}
    1 - p_i + \alpha \mathbf{1}\{p_i \le p_j\} \frac{1}{ntie}  & \text{if } p_i \le \bar{p}\\ 
    0  & \text{otherwise. } \\ 
\end{cases}
$$

In [9]:
def demand_function(p1, p2, i): 
    alpha = 0.5
    u1 = np.exp(1. - 1.0*p1 + alpha*(p1<p2) + 0.5*alpha*(p1==p2))
    u2 = np.exp(1. - 1.0*p2 + alpha*(p2<p1) + 0.5*alpha*(p1==p2))
    u0 = 1.0 
    
    denom = u0 + u1 + u2
    
    if i == 0: 
        return u1/denom 
    elif i == 1: 
        return u2/denom
    else: 
        raise Exception(f'Cannot return demand for player i={i}: must be 0 or 1.')

In [10]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
players_file_path = './players/'
tournament2 = Tournament(players_filepath=players_file_path, game=RepeatedBertrandGame, tournament_name='PriceComp')
print(tournament2)

np.random.seed(1337)
beta = 0.99 
tournament2.start_tournament(T=100, beta=beta, demand_function=demand_function, marginal_cost=1.0, price_range=(pmin, pmax), discount_factor=beta)
print(tournament2)
tournament_list.append(tournament2)

  0%|          | 0/10 [00:00<?, ?it/s]

Tournament ready with 5 players


100%|██████████| 10/10 [00:00<00:00, 97.64it/s]


Top placements are:
                   Name  Points
3  Undercutting Bastard     8.0
1               Randawg     6.0
4        Tit-for-tatter     3.0
2                  PMAX     3.0
0                  pmin     0.0
Finished tournament among 5 players, winner was: pmin


Show the total rankings of the game

In [12]:
tournament2.tournament_rank

,Name,Points
3,Undercutting Bastard,8.0
1,Randawg,6.0
4,Tit-for-tatter,3.0
2,PMAX,3.0
0,pmin,0.0


# Pure Bertrand

In [13]:
def demand_function(p1, p2, i): 
    alpha = 3.0 # punishment for being too expensive
    u1 = np.exp(1.-1.0*p1 - alpha*(p1>p2))
    u2 = np.exp(1.-1.0*p2 - alpha*(p2>p1))
    u0 = 1.0 
    
    denom = u0 + u1 + u2
    
    if i == 0: 
        return u1/denom 
    elif i == 1: 
        return u2/denom
    else: 
        raise Exception(f'Cannot return demand for player i={i}: must be 0 or 1.')

In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
players_file_path = './players/'
tournament3 = Tournament(players_filepath=players_file_path, game=RepeatedBertrandGame, tournament_name='PriceComp')
print(tournament3)

np.random.seed(1337)
beta = 0.99 
tournament3.start_tournament(T=100, beta=beta, demand_function=demand_function, marginal_cost=1.0, price_range=(pmin, pmax), discount_factor=beta)

tournament_list.append(tournament3)

# Show the total rankings of the game
tournament3.tournament_rank

  0%|          | 0/10 [00:00<?, ?it/s]

Tournament ready with 5 players


100%|██████████| 10/10 [00:00<00:00, 117.51it/s]


Top placements are:
                   Name  Points
3  Undercutting Bastard     8.0
1               Randawg     6.0
4        Tit-for-tatter     3.0
2                  PMAX     3.0
0                  pmin     0.0


,Name,Points
3,Undercutting Bastard,8.0
1,Randawg,6.0
4,Tit-for-tatter,3.0
2,PMAX,3.0
0,pmin,0.0


# Collect results

In [16]:
game_names = ['Pure Cournot', 'Pure Bertrand', 'Half-half']

# initialize dataframe
res = tournament_list[0].tournament_rank.rename(columns={'Points':game_names[0]}).set_index('Name')

# merge all other in list 
i = 1
for g in tournament_list[1:]: 
    res = res.join(g.tournament_rank
                   .rename(columns={'Points':game_names[i]}).set_index('Name')
                  )
    i += 1

In [17]:
res

,Pure Cournot,Pure Bertrand
Name,,
Undercutting Bastard,8.0,8.0
Randawg,6.0,6.0
Tit-for-tatter,3.0,3.0
PMAX,3.0,3.0
pmin,0.0,0.0


In [18]:
res.mean(1)

Name
Undercutting Bastard    8.0
Randawg                 6.0
Tit-for-tatter          3.0
PMAX                    3.0
pmin                    0.0
dtype: float64

In [19]:
res['Total'] = res.mean(1)

In [20]:
res.to_csv('results.csv')